## Arnoldi's methods

In [1]:
import LinearAlgebra
const la = LinearAlgebra
MAX_ITER = 100;

In [2]:
A = [4. -1 0 -1 0 0;
     -1 4 -1 0 -1 0;
     0 -1 4. 0 0 -1;
     -1 0 0. 4 -1 0;
     0 -1 0 -1 4 -1;
     0 0 -1 0. -1 4]

b = [0, 5, 0, 6, -2, 6.]

x0 = zeros(size(b));
# n
v = [1., 0, 2, 0, 1, 1];

### Arnoldi Gram-Schmidt
Input: $A\in\mathbb{R}^{n\times n}=[a_1,...,a_n], v\in\mathbb{R}^{n}$ any vector

In [3]:
function arnoldi(A:: Matrix, v:: Vector, m:: Int64)
    # Input: nxn matrix A, nx1 column vector v, and integer m
    # Output: (m + 1)xm matrix H and nx1 a nx(m + 1) matrix V
    n = size(A, 1)
    V = zeros((n, m + 1))
    H = zeros((m + 1, m))
    V[:, 1] = v/la.norm(v)
    for j = 1:m
        for i = 1:j
            H[i, j] = la.dot(A*V[:, j], V[:, i])
        end
        w = A*V[:, j] - V[:, 1:j]*H[1:j, j]
        H[j+1, j] = la.norm(w)
        if H[j+1, j] < eps()
            break
        end
        V[:, j+1] = w/H[j+1, j]
    end
    # delete last row of H and last column of V
    return H[1:m, :], V[:, 1:m]
end

arnoldi (generic function with 1 method)

In [4]:
m = 3
H, V = arnoldi(A, v, m);
display(H)
display(V)

3×3 Matrix{Float64}:
 3.14286  1.92195  -2.77556e-15
 1.92195  4.6472    1.18932
 0.0      1.18932   3.95556

6×3 Matrix{Float64}:
 0.377964   0.168563    0.289595
 0.0       -0.786629    0.191845
 0.755929   0.140469   -0.282287
 0.0       -0.393314    0.0959226
 0.377964  -0.0280939   0.750937
 0.377964  -0.421408   -0.475959

### Arnoldi-Modified Gram-Schmidt

In [ ]:
function arnoldi_modified(A:: Matrix, v:: Vector, m:: Int64)
    # Input: nxn matrix A, nx1 column vector v, and integer m
    # Output: (m + 1)xm matrix H and nx1 a nx(m + 1) matrix V
    n = size(A, 1)
    
    H = zeros((m + 1, m))
    V = zeros((n, m + 1))
    
    V[:, 1] = v/la.norm(v)
    for j = 1:m
        w = A*V[:, j]
        for i = 1:j
            H[i, j] = la.dot(w, V[:, i])
            w = w - H[i, j]*V[:, i] 
        end
        H[j+1, j] = la.norm(w)
        if H[j+1, j] < eps()
            break
        end
        V[:, j+1] = w/H[j+1, j]
    end
    # delete last row of H and last column of V
    return H[1:m, :], V[:, 1:m]
end

arnoldi_modified (generic function with 1 method)

In [ ]:
m = 3
H, V = arnoldi_modified(A, v, m)
display(H)
display(V)

3×3 Matrix{Float64}:
 3.14286  1.92195  -2.9976e-15
 1.92195  4.6472    1.18932
 0.0      1.18932   3.95556

6×3 Matrix{Float64}:
 0.377964   0.168563    0.289595
 0.0       -0.786629    0.191845
 0.755929   0.140469   -0.282287
 0.0       -0.393314    0.0959226
 0.377964  -0.0280939   0.750937
 0.377964  -0.421408   -0.475959

### Householder Arnoldi

In [ ]:
function householder_arnoldi(A:: Matrix, v:: Vector, m:: Int64)
    # Input: nxn matrix A, nx1 column vector v, and integer m < n
    # Output: nx(m+1) matrix H and an nxm orthonormal matrix V
    n = size(A, 1)
    I = 1.0 * Matrix(la.I, n, n)
    V = zeros((n, m + 1))
    H = zeros((n, m + 1))
    Z = zeros((n, m + 1))

    Z[:, 1] = v
    Q = I
    for j = 1:m + 1
        # w calculus
        val_sign = Z[j, j] >= 0.0 ? 1.0 : -1.0
        beta = val_sign * la.norm(Z[j:end, j])
        z = zeros(n)
        z[j] = beta - Z[j, j] + eps()
        for i = j + 1:n
            z[i] = -Z[i, j]
        end
        w = z / la.norm(z)
        # proyector
        P = I - 2 * w * w'
        # h_{j-1}
        H[:, j] = P * Z[:, j]
        # Pj*...*P2*P1
        Q = P * Q
        V[:, j] = Q'[:, j] # P1*P2...*Pj -> j-ésima
        if j <= m
            Z[:, j + 1] = Q * A * V[:, j]
        end
    end
    # delete the column 0 and return a (m+1)xm matrix H
    return H[1:m + 1, 2:m + 1], V
end

householder_arnoldi (generic function with 1 method)

In [ ]:
m = 4
H, V = householder_arnoldi(A, v, m)
show(IOContext(stdout, :limit => false), "text/plain", H)
display(V)

5×4 Matrix{Float64}:
  3.14286      -1.92195       0.0           2.22045e-16
 -1.92195       4.6472        1.18932      -2.77556e-16
 -2.77556e-16   1.18932       3.95556      -0.812257
  2.77556e-16  -1.07499e-16  -0.812257      3.67305
 -5.55112e-17  -6.55942e-18   3.09757e-16   0.114091

6×5 Matrix{Float64}:
 0.377964  -0.168563   -0.289595   -0.585252    0.390312
 0.0        0.786629   -0.191845    0.228787    0.540433
 0.755929  -0.140469    0.282287    0.128662    0.2502
 0.0        0.393314   -0.0959226  -0.699892   -0.310248
 0.377964   0.0280939  -0.750937    0.313906   -0.440352
 0.377964   0.421408    0.475959    0.0140224  -0.45036

Se verifica $AV_m = V_{m+1}H$

In [ ]:
isapprox(A*V[:, 1:m], V*H)

true

## Arnoldi’s Method for Linear Systems (FOM)

In [ ]:
function restarted_FOM(A:: Matrix, b:: Vector, x0:: Vector, m:: Int64, tol=1.0e-6)
    println("x[0] = $x0")
    for i=1:MAX_ITER
        r = b - A*x0
        beta = la.norm(r)
        v = r/beta
        H, V = arnoldi(A, v, m)
        # H = H[1:m, :]
        # V = V[:, 1:m]
        y = la.inv(H)[:, 1] * beta
        x = x0 + V*y
        println("x[$i] = $x")
        if la.norm(x - x0) < tol
            break
        end
        # updating for next step
        x0 = copy(x)
    end
end

restarted_FOM (generic function with 2 methods)

In [ ]:
m = 5
restarted_FOM(A, b, x0, m)

x[0] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
x[1] = [1.0000000000000004, 1.9999999999999998, 1.0000000000000004, 2.0, 1.0000000000000004, 2.0]
x[2] = [1.0, 1.9999999999999998, 1.0, 1.9999999999999998, 1.0, 2.0]


In [5]:
function e_vector(m:: Int64, k:: Int64)
    e = zeros(m)
    e[k] = 1.0
    return e
end

e_vector (generic function with 1 method)

In [6]:
function omega(H:: Matrix, i:: Int64)
    # Input: (m + 1) x m matrix H and column i
    # Output: (m + 1) x (m + 1) matrix O
    m = size(H, 2)
    O = 1.0 * Matrix(la.I, m + 1, m + 1) 
    hyp = sqrt(H[i, i]^2 + H[i + 1, i]^2)
    si = H[i + 1, i]/hyp
    ci = H[i, i]/hyp
    
    O[i, i] = ci
    O[i, i + 1] = si
    O[i + 1, i] = -si
    O[i + 1, i + 1] = ci
    return O 
end

omega (generic function with 1 method)

In [7]:
function givens(H:: Matrix, beta:: Vector)
    # Input: (m + 1) x m matrix H and (m + 1) x 1 vector beta
    # Output: (m + 1) x m matrix R, (m + 1) x (m + 1) matrix Q and (m + 1) x 1 vector gamma 
    m = size(H, 2)
    R = copy(H)
    for i = 1:m
        upper_omega = omega(R, i)
        beta = upper_omega * beta
        R = upper_omega * R
    end
    # delete last row, page 176 book 1
    return R[1:m, :], beta[1:m]
end

givens (generic function with 1 method)

In [9]:
function GMRES(A:: Matrix, b:: Vector, x0:: Vector, m:: Int64, tol=1.0e-5)
    # Solve Ax = b using the GMRES method
    # Input: n x n matrix A, n x 1 vector b,
    # initial approximation x0, integer m < n,
    # error tolerance tol
    println("x[0] = $x0")
    for i = 1:MAX_ITER
        r = b - A*x0
        V, H = arnoldi(A, r, m)
        beta = la.norm(r)
        # Solve the (m + 1)xm least-squares problem using Givens transformation
        R, g = givens(H, beta*e_vector(m + 1, 1))
        y = la.inv(R)*g
        x = x0 + V[:, 1:m]*y
        println("x[$i] = $x")
        if la.norm(x0 - x) < tol
            print("Number of iterations: $i")
            return nothing
        end
        x0 = copy(x)
    end
    print("There isn't convergence")
end

GMRES (generic function with 2 methods)

In [10]:
m = 4
GMRES(A, b, x0, m)

x[0] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
x[1] = [0.9999999999999999, 2.0, 0.9999999999999999, 2.0, 0.9999999999999999, 2.0]
x[2] = [1.0, 2.0, 1.0, 2.0, 1.0, 2.0]
Number of iterations: 2

### References
[1] Saad, Yousef. *Iterative methods for sparse linear systems*. Society for Industrial and Applied Mathematics, 2003.

[2] William Ford. *Numerical Linear Algebra with Applications Using MATLAB*, chapter 21.